In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")
ALPHAVANTAGE_API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

In [2]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client["robo_advisor"]  # Use the "robo_advisor" database
collection = db["historical_prices"]  # Use the "market_data" collection

In [3]:
# Fetch distinct tickers from MongoDB
tickers = db['historical_prices'].distinct("Asset")
print(f"Fetched {len(tickers)} tickers from MongoDB")

Fetched 71 tickers from MongoDB


In [4]:
# Initialize a list to store general information
data = []

In [ ]:
import yfinance as yf
import pandas as pd

# Fetch sector and industry information for each ticker
for ticker in tickers:
    try:
        # Download ticker info
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Append the sector and industry details
        data.append({
            "Ticker": ticker,
            "Short Name": info.get("shortName", "N/A"),
            "Long Name": info.get("longName", "N/A"),
            "Asset_Type": info.get("quoteType", "N/A"), 
            "Sector": info.get("sector", "N/A"),
            "Industry": info.get("industry", "N/A"),
            "Exchange": info.get("exchange", "N/A"),
            "Currency": info.get("currency", "N/A"),
            "Description": info.get("longBusinessSummary", "N/A")
        })
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        data.append({"Ticker": ticker, "Sector": "Error", "Industry": "Error", "Asset_Type": "Error"})

# Convert the data into a Pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

   Ticker                       Short Name  \
0    AAPL                       Apple Inc.   
1     AGG  iShares Core U.S. Aggregate Bon   
2     AMD     Advanced Micro Devices, Inc.   
3    AMGN                       Amgen Inc.   
4    AMZN                 Amazon.com, Inc.   
..    ...                              ...   
66    XLF  SPDR Select Sector Fund - Finan   
67    XLK  SPDR Select Sector Fund - Techn   
68    XLV  SPDR Select Sector Fund - Healt   
69    XLY  SPDR Select Sector Fund - Consu   
70     ZG               Zillow Group, Inc.   

                                            Long Name Asset_Type  \
0                                          Apple Inc.     EQUITY   
1                iShares Core U.S. Aggregate Bond ETF        ETF   
2                        Advanced Micro Devices, Inc.     EQUITY   
3                                          Amgen Inc.     EQUITY   
4                                    Amazon.com, Inc.     EQUITY   
..                                     

In [8]:
# Find N/A rows
df.isin(['N/A']).sum()

Ticker          0
Short Name      0
Long Name       0
Asset_Type      0
Sector         21
Industry       21
Exchange        0
Currency        0
Country        21
Description     0
Website        21
dtype: int64

In [6]:
# Find rows where any column contains "N/A"
na_rows = df[df.isin(['N/A']).any(axis=1)]

# Display rows with "N/A"
print(na_rows)

   Ticker                       Short Name  \
1     AGG  iShares Core U.S. Aggregate Bon   
9     BND   Vanguard Total Bond Market ETF   
19    DIA  SPDR Dow Jones Industrial Avera   
21    EEM  iShares MSCI Emerging Index Fun   
22    EFA            iShares MSCI EAFE ETF   
25    GLD                  SPDR Gold Trust   
27    HYG  iShares iBoxx $ High Yield Corp   
29    IEF  iShares 7-10 Year Treasury Bond   
32    IWM         iShares Russell 2000 ETF   
37    LQD  iShares iBoxx $ Investment Grad   
50    QQQ      Invesco QQQ Trust, Series 1   
51    SHY  iShares 1-3 Year Treasury Bond    
52    SPY                     SPDR S&P 500   
55    TLT  iShares 20+ Year Treasury Bond    
60    VEA  Vanguard FTSE Developed Markets   
64    VWO  Vanguard FTSE Emerging Markets    
65    XLE  SPDR Select Sector Fund - Energ   
66    XLF  SPDR Select Sector Fund - Finan   
67    XLK  SPDR Select Sector Fund - Techn   
68    XLV  SPDR Select Sector Fund - Healt   
69    XLY  SPDR Select Sector Fund

In [ ]:
import requests

# AlphaVantage API key
api_key = ALPHAVANTAGE_API_KEY

# Function to search for symbol metadata
def get_symbol_metadata(symbol):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&keywords={symbol}&apikey={api_key}"
    response = requests.get(url).json()
    
    # Extract relevant metadata
    best_match = response.get("bestMatches", [])
    if best_match:
        return {
            "Ticker": best_match[0].get("1. symbol", "N/A"),
            "Asset Class": best_match[0].get("3. type", "N/A")  # Indicates if it's an ETF, stock, etc.
        }
    return {"Ticker": symbol, "Asset Class": "Unknown"}


AssetType
Description
sector_dfIndustry
Country
Exchange
